# Electron Splits

In [1]:
%load_ext line_profiler
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from misc import *
from notation import *
from IPython.display import display, Math, Latex
# from joblib import Parallel, delayed
# import multiprocessing
# num_cores = multiprocessing.cpu_count()
import time

Reloading /Volumes/GoogleDrive/My Drive/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


In [64]:
class Electrons():
    def __init__(self, e_string):
        self.e_string = e_string
        self.n = int(e_string[0])
        self.l = l_from_lett_to_num[e_string[1]]
        self.num_electrons = int(e_string[2])
    def __repr__(self):
        return self.e_string
class MultiElectronConfig():
    def __init__(self, config_string):
        self.config_string = config_string
        self.electrons = [Electrons(e_config) for e_config in config_string.split(',')]
    def __repr__(self):
        return self.config_string


Here each final configuration is saved as a sequence of irrep symbols of length N, where the first N-1 correspond to the sequence of irreps as they were sequentially added and where the final symbol is the character that that combination may have.

At each stage, grab all the current irrep characters of the current combinations, to each of those add either of the single electron irreps, figure out what may come out of that, and then append these to the sequence of single electron irreps for each previous combination thus far.

In [378]:
'''
Given an MultiElectronConfiguration and a crystallographic point group
This function determines how the level will split under a potential with
the given symmetry
'''

multiconf = MultiElectronConfig('3d3')
group_label = 'O'
group = CPGs.get_group_by_label(group_label)
electrons_in_shells = multiconf.electrons

'''
How to build the n-electron set of irreps by iterating 
over pairs?

For each Electrons
-> take the value of l and determine their corresponding single-electron splitting
-> then, using the number of electrons, use mix the single-electron irreps to obtain
-> multi-electron irreps
'''

single_electron_reductions = []
electrons = electrons_in_shells[0]
l = electrons.l
group_irreps = group.irrep_labels
num_electrons = electrons.num_electrons
single_electron_split = l_splitter(group_label, l)
product_table_dict = group.product_table.odict
single_electron_irreps = list(single_electron_split.dict.keys())
multi_combos = [[c,c] for c in single_electron_irreps]

for _ in range(num_electrons-1):
    next_multi_combos = []
    for combo in multi_combos:
        current_irrep = combo[-1]
        for new_electron in single_electron_irreps:
            duo = (current_irrep, new_electron)
            branches = product_table_dict[(duo)]
            for branch in branches:
                next_multi_combos.append(combo[:-1]+[new_electron,branch])
    multi_combos = list(next_multi_combos)

final_characters = list(map(lambda x: x[-1], multi_combos))
multiplicities = [{ir: final_characters.count(ir)} for ir in group_irreps]

final_inequiv_combos = [list(map(lambda x: sp.Symbol(sp.latex(x).lower()), combo[:-1]))+[combo[-1]] for combo in multi_combos]
final_inequiv_combos = sp.Matrix(final_inequiv_combos)
final_equiv_combos = [(frozenset(map(lambda x: sp.Symbol(sp.latex(x).lower()), combo[:-1])),combo[-1]) for combo in multi_combos]

unique_combos = frozenset(list(map(lambda x: x[0], final_equiv_combos)))
final_combo_collection = {uc:[x[-1] for x in final_equiv_combos if x[0] == uc] for uc in unique_combos}
final_combo_collection = {k:{ir:v.count(ir) for ir in single_electron_irreps if v.count(ir) !=0} for k,v in final_combo_collection.items()}


In [381]:
display(Math(group.product_table.pretty_parse()))

<IPython.core.display.Math object>

In [379]:
final_combo_collection

{frozenset({t_{2}}): {E: 2, T_2: 4},
 frozenset({e, t_{2}}): {E: 6, T_2: 12},
 frozenset({e}): {E: 3}}

In [366]:
unique_combos = frozenset(list(map(lambda x: x[0], final_equiv_combos)))
final_combo_collection = {uc:[x[-1] for x in final_equiv_combos if x[0] == uc] for uc in unique_combos}

In [357]:
frozenset(list(map(lambda x: x[0], final_equiv_combos)))

frozenset({frozenset({t_{2}}), frozenset({e}), frozenset({e, t_{2}})})

In [352]:
from functools import reduce

In [314]:
reduce(sp.Mul,[1,2,3])

6

If the individual electrons are equivalent then the ordering of the irreps should not matter.
For example {E,E,E,A1,T1} should be considered equivalent to {A1,E,E,E,T1}.

For inequivalent electrons the total number of states at the end should equal, for d-electrons, (2*(2l+1))^(number of electrons), since this Hilbert space is simply the one spanned by the all the combinations of the 2*(2l+1) states with the given l and spin 1/2.

In [ ]:
# class AngularMomentumNotation():
#     def __init__(self):
#         pass
#     spectroscopic_alphabet = 'spdfghiklmnoqrtuv'
#     l_from_lett_to_num = OrderedDict([(cls.spectroscopic_alphabet[i],i) for i in range(len(cls.spectroscopic_alphabet))])
#     l_from_num_to_lett = OrderedDict([(i,cls.spectroscopic_alphabet[i]) for i in range(len(cls.spectroscopic_alphabet))])
#     @classmethod
#     def switch(cls, origin):
#         '''
#         If given a string it returns an int,
#         if given an int it returns a char.
#         '''
#         # if not hasattr(cls, 'l_from_lett_to_num'):
#         #     cls,l_from_lett_to_num = cls.
#         if isinstance(origin,int):
#             return cls.l_from_num_to_lett[origin]
#         elif isinstance(origin, str):
#             assert len(origin) == 1, "only one character allowed"
#             assert origin in cls.spectroscopic_alphabet
#             return cls.l_from_lett_to_num[origin]

In [ ]:
spectroscopic_alphabet = 'spdfghiklmnoqrtuv'
l_from_lett_to_num = OrderedDict([(spectroscopic_alphabet[i],i) for i in range(len(spectroscopic_alphabet))])
l_from_num_to_lett = OrderedDict([(i,spectroscopic_alphabet[i]) for i in range(len(spectroscopic_alphabet))])

In [83]:
class AngularMomentumNotation():
    spectroscopic_alphabet = 'spdfghiklmnoqrtuv'
    l_from_lett_to_num = OrderedDict([('spdfghiklmnoqrtuv'[i],i) for i in range(len(spectroscopic_alphabet))])
    l_from_num_to_lett = OrderedDict([(i,spectroscopic_alphabet[i]) for i in range(len(spectroscopic_alphabet))])
    @classmethod
    def switch(cls, origin):
        '''
        If given a string it returns an int,
        if given an int it returns a char.
        '''
        if isinstance(origin,int):
            return cls.l_from_num_to_lett[origin]
        elif isinstance(origin, str):
            assert len(origin) == 1, "only one character allowed"
            return cls.l_from_lett_to_num[origin]

